# Rusty Bargain
# Descripcion del proyecto

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
import catboost as cb
import xgboost as xgb

In [2]:
df = pd.read_csv('/datasets/car_data.csv')

# Exploramos por separado cada una de las variables buscando:

Datos nulos

Datos inconsistentes

In [3]:
print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [4]:
df['DateCrawled'] = pd.to_datetime(df['DateCrawled'])
df['LastSeen'] = pd.to_datetime(df['LastSeen'])

In [5]:
display(df.head())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:00,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,2016-07-04 03:16:00
1,2016-03-24 10:58:00,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,2016-07-04 01:46:00
2,2016-03-14 12:52:00,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,2016-05-04 12:47:00
3,2016-03-17 16:54:00,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,2016-03-17 17:40:00
4,2016-03-31 17:25:00,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,2016-06-04 10:17:00


In [6]:
print(df.isna().sum())

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Mileage                  0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64


In [7]:
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']
for col in categorical_columns:
    df[col] = df[col].fillna('unknown')

In [8]:
print(df.isna().sum())

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64


In [9]:
print(df)

               DateCrawled  Price  VehicleType  RegistrationYear  Gearbox  \
0      2016-03-24 11:52:00    480      unknown              1993   manual   
1      2016-03-24 10:58:00  18300        coupe              2011   manual   
2      2016-03-14 12:52:00   9800          suv              2004     auto   
3      2016-03-17 16:54:00   1500        small              2001   manual   
4      2016-03-31 17:25:00   3600        small              2008   manual   
...                    ...    ...          ...               ...      ...   
354364 2016-03-21 09:50:00      0      unknown              2005   manual   
354365 2016-03-14 17:48:00   2200      unknown              2005  unknown   
354366 2016-05-03 19:56:00   1199  convertible              2000     auto   
354367 2016-03-19 18:57:00   9200          bus              1996   manual   
354368 2016-03-20 19:41:00   3400        wagon              2002   manual   

        Power        Model  Mileage  RegistrationMonth  FuelType  \
0      

# Si existen datos inconsistentes:

Vamos a transformarlos en nan e imputarlos.

In [11]:
Q1_price = df['Price'].quantile(0.25)
Q3_price = df['Price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

Q1_power = df['Power'].quantile(0.25)
Q3_power = df['Power'].quantile(0.75)
IQR_power = Q3_power - Q1_power

df = df[(df['Price'] >= Q1_price - 1.5 * IQR_price) & (df['Price'] <= Q3_price + 1.5 * IQR_price)]
df = df[(df['Power'] >= Q1_power - 1.5 * IQR_power) & (df['Power'] <= Q3_power + 1.5 * IQR_power)]

print(df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 330273 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        330273 non-null  datetime64[ns]
 1   Price              330273 non-null  int64         
 2   VehicleType        330273 non-null  object        
 3   RegistrationYear   330273 non-null  int64         
 4   Gearbox            330273 non-null  object        
 5   Power              330273 non-null  int64         
 6   Model              330273 non-null  object        
 7   Mileage            330273 non-null  int64         
 8   RegistrationMonth  330273 non-null  int64         
 9   FuelType           330273 non-null  object        
 10  Brand              330273 non-null  object        
 11  NotRepaired        330273 non-null  object        
 12  DateCreated        330273 non-null  object        
 13  NumberOfPictures   330273 non-null  int64   

In [12]:
print(df.isna().sum())

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64


## Entrenamiento del modelo 

In [13]:
X = df.drop(['Price', 'RegistrationMonth', 'DateCrawled', 'RegistrationYear', 'LastSeen', 'DateCreated', 'PostalCode'], axis=1)
y = df['Price']

In [14]:
X = pd.get_dummies(X, drop_first=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Exploracion de parametros

In [17]:
param_grid = {'max_depth': [3, 4, 5],
              'min_samples_split': [2, 3, 4, 5]}

regressor = DecisionTreeRegressor(random_state=1234)

tree_gs = GridSearchCV(regressor,
                       param_grid=param_grid,
                       scoring='neg_root_mean_squared_error',
                       cv=3)

tree_gs.fit(X_train, y_train)

pred_tree = tree_gs.predict(X_test)
test_metric = mean_squared_error(y_test, pred_tree, squared=False)

print('Best parameters:', tree_gs.best_params_)
print('RMSE:', test_metric)


Best parameters: {'max_depth': 5, 'min_samples_split': 2}
RMSE: 2432.8209479368516


## Definimos los modelos

In [18]:
models = {
    'Random Forest': RandomForestRegressor(random_state=1234),
    'LGBM': lgb.LGBMRegressor(random_state=1234),
    'Catboost': cb.CatBoostRegressor(random_state=1234, silent=True),
    'XGBoost': xgb.XGBRegressor(random_state=1234, verbosity=0),
}

In [19]:
X_train


,Power,Mileage,NumberOfPictures,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,VehicleType_suv,VehicleType_unknown,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_unknown,NotRepaired_yes
142514,69,90000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1615,88,150000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
225516,105,150000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72634,50,150000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
223761,0,150000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128675,240,90000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
278197,97,30000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
141621,55,100000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
157667,135,125000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Exploracion de los mejores parametros

In [20]:

models = {
    'Random Forest': RandomForestRegressor(random_state=1234),
    'LGBM': lgb.LGBMRegressor(random_state=1234),
    'Catboost': cb.CatBoostRegressor(random_state=1234, silent=True),
    'XGBoost': xgb.XGBRegressor(random_state=1234, verbosity=0),
}

param_grid_b = {'max_depth': [3, 4],
                'n_estimators': [25, 50]}

param_grids = {
    'Random Forest': param_grid_b,
    'LGBM': param_grid_b,
    'Catboost': param_grid_b,
    'XGBoost': param_grid_b
}

best_params = {}
test_metrics = {}

for model_name, model in models.items():
    print(f"Training {model_name}...")
    gs = GridSearchCV(model,
                      param_grid=param_grids[model_name],
                      scoring='neg_root_mean_squared_error',
                      cv=3)
    gs.fit(X_train, y_train)
    
    pred = gs.predict(X_test)
    test_metric = mean_squared_error(y_test, pred, squared=False)
    
    best_params[model_name] = gs.best_params_
    test_metrics[model_name] = test_metric
    
    print(f"{model_name} - Best parameters:", gs.best_params_)
    print(f"{model_name} - RMSE on the test set:", test_metric)
    print("============================================")

Training Random Forest...
Random Forest - Best parameters: {'max_depth': 4, 'n_estimators': 25}
Random Forest - RMSE on the test set: 2491.8900846988636
Training LGBM...
LGBM - Best parameters: {'max_depth': 4, 'n_estimators': 50}
LGBM - RMSE on the test set: 2219.058876453273
Training Catboost...
Catboost - Best parameters: {'max_depth': 4, 'n_estimators': 50}
Catboost - RMSE on the test set: 2120.5640150220015
Training XGBoost...
XGBoost - Best parameters: {'max_depth': 4, 'n_estimators': 50}
XGBoost - RMSE on the test set: 2122.86152365571


## Análisis de los resultados

In [21]:
pd.Series(test_metrics).reset_index(name='test metric').rename(columns={'index':'model'}) 

,model,test metric
0,Random Forest,2491.890085
1,LGBM,2219.058876
2,Catboost,2120.564015
3,XGBoost,2122.861524


## Conclusiones

Despues de realizar optimización de hiperparámetros, donde evaluamos cuatro modelos diferentes: Random Forest, LGBM, Catboost y XGBoost. Estos se compararon en terminos de la metrica de prueba RMSE para medir la precision de las predicciones. En conclusion, el proyecto ha arrojado que Catboost, con hiperparmetros optimizados de max_depth 4 y n_estimators 50, nos ofrece la mejor calidad en cuestion de predicción con un rendimiento rápido y un tiempo de entrenamiento razonable. Esto brinda a Rusty Bargain una forma confiable para determinar de manera efectiva el valor de sus vehiculos